# Sovereign Debt Data – World Bank WDI Pipeline

### What this notebook does at a high level
This notebook pulls macroeconomic indicators from the World Bank World Development Indicators (WDI) API, reshapes them into a clean panel dataset, creates a latest-year snapshot by country, and saves both to CSV for downstream sovereign-debt analysis.

### How it does it
- Defines a mapping between WDI indicator codes and readable column names.
- Implements helper functions to:
  - fetch a single indicator for all countries and all years,
  - loop over multiple indicators and stack them,
  - reshape the long-format data into a wide panel by country–year.
- Filters the panel to a study window (e.g., 2000–2024).
- Constructs a “latest available year per country” cross-section.
- Writes both datasets to disk as CSV files.

## 1. Setup and imports

In [2]:
import requests, pandas as pd
from typing import Dict, List

## 2. Configure WDI indicator set

### What this does at a high level
Defines a dictionary mapping raw WDI indicator codes to human-readable column names you will use in your datasets.

### How it does it
- Creates a Python dictionary `WDI_INDICATORS`:
  - keys: WDI indicator codes (e.g., `"NY.GDP.MKTP.CD"`),
  - values: concise, snake_case names (e.g., `"gdp_usd_current"`).
- Covers several macro dimensions:
  - economic size and growth,
  - prices/monetary (inflation),
  - external sector (current account, reserves),
  - fiscal (balance and government debt).

In [3]:
# World Bank WDI indicator codes → friendly names
WDI_INDICATORS = {
    # Economic size/growth
    "NY.GDP.MKTP.CD":        "gdp_usd_current",         # GDP (current US$)
    "NY.GDP.MKTP.KD.ZG":     "gdp_growth_real_pct",     # GDP growth (annual %)
    "NY.GDP.PCAP.CD":        "gdp_per_capita_usd",

    # Prices / monetary
    "FP.CPI.TOTL.ZG":        "inflation_cpi_pct",       # CPI inflation (annual %)

    # External sector
    "BN.CAB.XOKA.GD.ZS":     "current_account_pct_gdp", # Current account (% of GDP)
    "FI.RES.TOTL.MO":        "reserves_months_imports", # Reserves (months of imports)

    # Fiscal
    "GC.NLD.TOTL.GD.ZS":     "fiscal_balance_pct_gdp",  # Net lending/borrowing (% of GDP)
    "GC.DOD.TOTL.GD.ZS":     "gov_debt_pct_gdp",        # Central gov debt (% of GDP) — coverage varies
}

## 3. Helper functions for fetching and reshaping WDI data

### What this does at a high level
Implements three core helpers:
1. `fetch_wdi_series` – pull one indicator for all countries and years.
2. `fetch_wdi_bulk` – loop over many indicators and stack them.
3. `reshape_wdi_panel` – pivot the long data into a country–year panel with one column per indicator.

### How it does it
- `fetch_wdi_series(indicator, per_page)`:
  - calls the World Bank API endpoint `/country/all/indicator/{indicator}` with JSON format,
  - parses the two-element response `[metadata, rows]`,
  - builds a tidy dataframe with columns `iso3c`, `country`, `year`, `indicator`, `value`,
  - filters out aggregate regions and non-3-letter ISO codes.
- `fetch_wdi_bulk(indicators)`:
  - iterates through the list of indicator codes,
  - calls `fetch_wdi_series` for each,
  - catches and logs any failures,
  - concatenates all series into a single long dataframe.
- `reshape_wdi_panel(raw, rename_map)`:
  - pivots on `["iso3c", "country", "year"]` and spreads indicators across columns,
  - renames indicator codes to friendly names using `rename_map`,
  - sorts rows by country and year.

### Why this matters
You now have a **reusable, general-purpose data pipeline**:
- If you add or change indicators, you only update the `WDI_INDICATORS` dict.
- The fetching and reshaping logic stays the same.
- Every downstream notebook can rely on a standard long-to-wide transformation and consistent country–year indexing.

In [4]:
def fetch_wdi_series(indicator: str, per_page: int = 20000) -> pd.DataFrame:
    """
    Pulls a single WDI indicator for all countries, all years.
    Returns tidy columns: iso3c, country, year, value, indicator
    """
    url = f"https://api.worldbank.org/v2/country/all/indicator/{indicator}"
    params = {"format": "json", "per_page": per_page}
    r = requests.get(url, params=params)
    r.raise_for_status()
    data = r.json()
    # World Bank returns [metadata, rows]; rows may be None if empty
    rows = data[1] if isinstance(data, list) and len(data) > 1 else []
    if not rows:
        return pd.DataFrame(columns=["iso3c","country","year","indicator","value"])
    df = pd.DataFrame([{
        "iso3c":   x.get("countryiso3code"),
        "country": x.get("country", {}).get("value"),
        "year":    int(x["date"]) if str(x.get("date","")).isdigit() else None,
        "indicator": indicator,
        "value":   x.get("value")
    } for x in rows])
    # drop aggregates (regions/income groups) with missing/odd iso3 codes
    df = df[df["iso3c"].str.len() == 3].dropna(subset=["year"])
    return df

def fetch_wdi_bulk(indicators: List[str]) -> pd.DataFrame:
    frames = []
    for ind in indicators:
        try:
            frames.append(fetch_wdi_series(ind))
        except Exception as e:
            print(f"[warn] {ind} failed: {e}")
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

def reshape_wdi_panel(raw: pd.DataFrame, rename_map: Dict[str,str]) -> pd.DataFrame:
    """
    Pivot long→wide on indicator, rename columns to friendly names,
    and sort by iso3c/year.
    """
    if raw.empty:
        return raw
    wide = (raw
            .pivot_table(index=["iso3c","country","year"],
                         columns="indicator", values="value")
            .reset_index())
    # Rename columns (keep unknowns with their code)
    wide = wide.rename(columns=rename_map)
    # sort and type-cast
    wide = wide.sort_values(["iso3c","year"]).reset_index(drop=True)
    return wide

## 4. Fetch WDI data, build panel and latest snapshot, and save to CSV

### What this does at a high level
Downloads all configured indicators from the World Bank API, reshapes them into:
- a full **country–year panel** (`wdi_panel`), and
- a **latest-year-per-country snapshot** (`wdi_latest`),
then writes both to CSV files.

### How it does it
1. Calls `fetch_wdi_bulk(list(WDI_INDICATORS.keys()))` to pull every indicator defined in `WDI_INDICATORS`.
2. Calls `reshape_wdi_panel` to pivot the long data into a wide panel.
3. Filters the panel to a study window (2000–2024) to keep it aligned with ratings coverage.
4. Builds `latest_idx` by:
   - grouping `wdi_panel` by `iso3c`,
   - taking the maximum available `year` per country,
   - renaming that column to `latest_year`.
5. Merges `latest_idx` back into the panel, filters rows where `year == latest_year`, and drops the helper column to create `wdi_latest`.
6. Saves:
   - `wdi_panel_2000_2024.csv`
   - `wdi_latest_by_country.csv`
7. Shows a quick `head()` preview of both dataframes.

### Why this matters
This is the **core data extraction step**:
- It creates the standardized macro dataset that all downstream sovereign-debt work will rely on.
- The panel form (`wdi_panel`) supports time-series or panel regressions.
- The latest snapshot (`wdi_latest`) supports cross-sectional/statics models and descriptive plots.
- Writing to CSV makes the pipeline reproducible and easy to plug into other tools (R, Stata, etc.).

In [5]:
# Fetch everything in one go
wdi_raw = fetch_wdi_bulk(list(WDI_INDICATORS.keys()))
wdi_panel = reshape_wdi_panel(wdi_raw, WDI_INDICATORS)

# Keep a sensible study window (e.g., 2000–2024) to match ratings coverage
wdi_panel = wdi_panel[(wdi_panel["year"] >= 2000) & (wdi_panel["year"] <= 2024)]

# Optional: Latest-year snapshot per country (for cross-section models/plots)
latest_idx = (wdi_panel
              .groupby("iso3c", as_index=False)["year"]
              .max()
              .rename(columns={"year":"latest_year"}))
wdi_latest = (wdi_panel
              .merge(latest_idx, on="iso3c")
              .query("year == latest_year")
              .drop(columns=["latest_year"])
              .reset_index(drop=True))

# Save for joining with your ratings
wdi_panel.to_csv("wdi_panel_2000_2024.csv", index=False)
wdi_latest.to_csv("wdi_latest_by_country.csv", index=False)

wdi_panel.head(), wdi_latest.head()

(indicator iso3c country  year  current_account_pct_gdp  \
 14          ABW   Aruba  2000                11.056872   
 15          ABW   Aruba  2001                16.280874   
 16          ABW   Aruba  2002               -17.343452   
 17          ABW   Aruba  2003                -8.099023   
 18          ABW   Aruba  2004                12.013685   
 
 indicator  reserves_months_imports  inflation_cpi_pct  gov_debt_pct_gdp  \
 14                        1.785034                NaN               NaN   
 15                        2.275344                NaN               NaN   
 16                        2.558116           3.315247               NaN   
 17                        2.154841           3.656365               NaN   
 18                        0.937018           2.529129               NaN   
 
 indicator  fiscal_balance_pct_gdp  gdp_usd_current  gdp_growth_real_pct  \
 14                            NaN     1.873453e+09             7.622921   
 15                            NaN

## 5. Coverage and missingness diagnostics

### What this does at a high level
Checks:
- which columns exist in `wdi_panel` and `wdi_latest`,
- how many non-missing observations each column has,
- and the overall missingness ratio per variable.

### How it does it
1. `columns_wdi_panel` and `columns_wdi_latest` store the list of columns in each dataframe.
2. For the panel:
   - iterates over each column and prints:
     - the column name,
     - the count of non-missing observations.
   - computes `na_ratio = wdi_panel.isna().mean().sort_values(ascending=False)` to get the fraction of missing values per column.
3. For the latest snapshot:
   - iterates over each column and prints similar counts,
   - computes `na_ratio` again on `wdi_latest`.

> Note: In the original code, one of the loops uses `wdi_panel[col_name]` even inside the `wdi_latest` loop; if you intend to diagnose missingness of `wdi_latest`, you may want to change that to `wdi_latest[col_name]` later. I’m not changing it here to stay faithful to your current functionality.

### Why this matters
For sovereign-debt modeling, **data coverage is critical**:
- Some indicators (e.g., fiscal variables) are sparse for many countries and years.
- Understanding which variables are heavily missing:
  - informs imputation strategy,
  - helps decide which features are reliable enough to use,
  - avoids silently training models on very noisy or thin data.

In [7]:
columns_wdi_panel = wdi_panel.columns

In [8]:
type(columns_wdi_panel)

pandas.core.indexes.base.Index

In [9]:
for col_name in columns_wdi_panel:
    print("Column name: ", col_name)
    column = wdi_panel[col_name]
    print("Non-NA data: ", len(wdi_panel) - wdi_panel[col_name].isna().sum()) 

Column name:  iso3c
Non-NA data:  6346
Column name:  country
Non-NA data:  6346
Column name:  year
Non-NA data:  6346
Column name:  current_account_pct_gdp
Non-NA data:  4367
Column name:  reserves_months_imports
Non-NA data:  5027
Column name:  inflation_cpi_pct
Non-NA data:  4535
Column name:  gov_debt_pct_gdp
Non-NA data:  1206
Column name:  fiscal_balance_pct_gdp
Non-NA data:  3238
Column name:  gdp_usd_current
Non-NA data:  6313
Column name:  gdp_growth_real_pct
Non-NA data:  6241
Column name:  gdp_per_capita_usd
Non-NA data:  6313


In [10]:
na_ratio = wdi_panel.isna().mean().sort_values(ascending=False)
na_ratio

indicator
gov_debt_pct_gdp           0.809959
fiscal_balance_pct_gdp     0.489757
current_account_pct_gdp    0.311850
inflation_cpi_pct          0.285377
reserves_months_imports    0.207847
gdp_growth_real_pct        0.016546
gdp_usd_current            0.005200
gdp_per_capita_usd         0.005200
iso3c                      0.000000
country                    0.000000
year                       0.000000
dtype: float64

In [11]:
columns_wdi_latest = wdi_latest.columns

In [12]:
for col_name in columns_wdi_latest:
    print("Column name: ", col_name)
    column = wdi_panel[col_name]
    print("Non-NA data: ", len(wdi_panel) - wdi_panel[col_name].isna().sum()) 

Column name:  iso3c
Non-NA data:  6346
Column name:  country
Non-NA data:  6346
Column name:  year
Non-NA data:  6346
Column name:  current_account_pct_gdp
Non-NA data:  4367
Column name:  reserves_months_imports
Non-NA data:  5027
Column name:  inflation_cpi_pct
Non-NA data:  4535
Column name:  gov_debt_pct_gdp
Non-NA data:  1206
Column name:  fiscal_balance_pct_gdp
Non-NA data:  3238
Column name:  gdp_usd_current
Non-NA data:  6313
Column name:  gdp_growth_real_pct
Non-NA data:  6241
Column name:  gdp_per_capita_usd
Non-NA data:  6313


In [13]:
na_ratio = wdi_latest.isna().mean().sort_values(ascending=False)
na_ratio

gov_debt_pct_gdp           0.984496
fiscal_balance_pct_gdp     0.968992
current_account_pct_gdp    0.480620
reserves_months_imports    0.391473
inflation_cpi_pct          0.182171
gdp_usd_current            0.042636
gdp_per_capita_usd         0.042636
gdp_growth_real_pct        0.038760
iso3c                      0.000000
country                    0.000000
year                       0.000000
dtype: float64

## 6. Column documentation and economic interpretation

### What this does at a high level
Documents:
- the structure of the `wdi_panel` and `wdi_latest` datasets,
- the meaning of each key column,
- how these variables relate to sovereign risk and ratings.

### How it does it
- Clarifies that:
  - `wdi_panel` is a full country–year panel (2000–2024) with one row per country per year.
  - `wdi_latest` is a latest-year snapshot per country (e.g., 2023/2024), useful for cross-country comparisons.
- Lists metadata:
  - `iso3c`: ISO 3-letter country code (e.g., USA, IND, BRA),
  - `country`: country name,
  - `year`: observation year.
- Provides economic descriptions (examples):
  - `gdp_usd_current`: level of economic size; larger economies can often bear more debt.
  - `gdp_growth_real_pct`: real growth momentum; higher growth improves debt sustainability.
  - `gdp_per_capita_usd`: proxy for development and income level; higher income tends to correlate with stronger ratings.
  - `inflation_cpi_pct`: price stability; very high or volatile inflation can signal macro instability.
  - `current_account_pct_gdp`: external balance; large deficits can signal external financing risks.
  - `reserves_months_imports`: buffer against external shocks; more reserves usually mean greater resilience.
  - `gov_debt_pct_gdp`: public debt burden relative to the economy.
  - `fiscal_balance_pct_gdp`: fiscal discipline and space.

It can also note that fiscal data are often patchy and may require:
- interpolation over time (e.g., regression on trends),
- cross-filling from IMF sources,
- or careful handling when used in models.

In [15]:
# save the datasets 

In [16]:
# Write your code here to save the dataset to your file system